# How to use the BT-Smart-Controller in Python

This notebook shows you, how to use the btsmart library to easily access the Fischertechnik bt-smart controller.

## The Scenario

In order to play with the controller, we build the following simple scenario:

A button, a lamp and a motor elements are attached to the bt-smart controller (button on input1, lamp on output1 and motor on output2).

When the button is pushed (contact closed), the lamp should be turned on and the motor shoul run for a short time.
When the button is released (contact open), the lamp should be turned off.

Basically, that is all we need to also build mor complex scenarios such a a light barrier for cars, triggering a traffic light or opening a gate. but we will get back to that later on.


## Install the required libraries

The first thing we need to do is make sure that the required btsmart library and the according dependncies are in stalled. You can use the following code to install the library.

In [ ]:
!pip install -i https://test.pypi.org/simple/ btsmart==0.1.6

## Start coding

Now that we have everything in place, we can start to write the program.

### 1. Import the required modules and classes

First, we have to import the libraries and required classes.
These are:
- asyncio
  this library contains everything we need to handle asynchronuous programs, which means
  programs that are not purely sequences, but may run in parallel (e.g. a light may blink while a motor turns)
- btsmart
  this is the library containing the btsmart-related classes, which are:
  - BTSmartController
    this class helps us to connect and manipulate the btsmart controller. It allows us to measure or react on 
    input value changes or to set output values.
    However, this class is still pretty "technical", so we use some additionaly classes...
  - Button
    this class represents a button (switch) attached to an input of the controller. It triggers events when the 
    button is pressed or when it is released, so we can use it to simply react on these events.
  - Dimmer
    this class basically sets the output level to a value between 0 and 100 in order to regulate the light 
    intensity, given that the output is attached to a lamp. It also provides means for blinking.
  - MotorXM
    this class provides functions to start, stop or run the attached motor for a certain amount of time with
    a given rpm value rather than using the values specified by the underlying BLE-protocol used by the controller
    

In [ ]:
# we need asyncio
import asyncio

# from the btsmart library, we need to access the controller class
from btsmart import BTSmartController, LEDMode, InputMode

# and as described in the scenario, we use some electronis parts
from btsmart import Button, Dimmer, MotorXM

### 2. Create objects that represent the electronical parts

In this step, we create objects that represent the electronical objects attached to our physical scenario.

In [ ]:
btn = Button()
dim = Dimmer()
motor = MotorXM()

### 3. Define the logic

As described above we want to react on input events. We do this by defining two functions
- 'button_pressed' should be called when the according input levels on the controller change in a way that
  we "close" the cirquit meaning,  the resistance goes down.
  In this case, we print out a message and start two actions in parallel (turn on the light and run the motor)
- 'button_released' should be called when the according input level changes in the opposite direction, meaning
  we open the cirquit.
  In this case, we show a message and turn off the light.

After defining the functions, we attach them to the button-object 'btn'

In [ ]:
async def button_pressed():
    #print("Button pressed")
    await asyncio.gather(                    # we start two different things at once...
        btSmart.set_led(LEDMode.BLUE),       # - we change the led color to blue
        dim.blink(time=0.05,count=3),        # - we turn on the light (50 means full output)
        motor.run_at(speed=150, time=0.5)    # - we let the motor run for half a second at 50 rpm
    )                                        # ... and wait until both operations are done

async def button_released():
    #print("Button released")
    await btSmart.set_led(LEDMode.ORANGE)   # change to orange light

btn.on_press(button_pressed)
btn.on_release(button_released)


## Run the program

Until now the controller is not linked to our code and we need to connect to it using bluetooth.
For this, when you run the following code, you need to press the "connect"-button on the BT-Smart.

In [ ]:
btSmart: BTSmartController = None
try:
    btSmart = await BTSmartController.discover()
except Exception as e:
    print(e)
    btSmart = None

if btSmart is None:
    print("No controller found - please press the connect button on the controller before running this code")

If we connected to the controller, we can attach our logical elements to it and that should do the magic.

In [ ]:
btn.attach(btSmart, 1)   # we attach the 'button' to input 1 of the controller
dim.attach(btSmart, 1)   # we attach the 'dimmer' to output 1 of the controller
motor.attach(btSmart, 2) # we attach the 'motor'  to output 2 of the controller

await btSmart.connect()  # connect to the controller and let it do the work


Thats's it - try to figure out what happens when you change the functions above and have fun.

In [ ]:
print(await btSmart.get_input_value(1, InputMode.RESISTANCE))